In [ ]:
import math
import bisect

import numpy
import scipy.optimize
import sympy

In [ ]:
sympy.init_printing()

In [ ]:
x = sympy.Symbol("x")

In [ ]:
def mat_integer_direct(num, deg):
    mat = numpy.fromfunction(lambda x, y: y**x, (deg+1, num+1), dtype=numpy.int64)
    int_ = numpy.fromfunction(lambda x: num**(x+1) / (x+1), (deg+1,), dtype=numpy.int64)
    return mat, int_

In [ ]:
def mat_interval_direct(num, deg):
    mat = numpy.fromfunction(lambda x, y: (2.0 * y / num - 1.0)**x, (deg+1, num+1))
    int_ = numpy.fromfunction(lambda x: 2.0 / (x+1), (deg+1,))
    int_[1::2] = 0.0
    return mat, int_

In [ ]:
cheb_list = [sympy.Integer(1), x]
cheb_int = [2.0, 0.0]

def mat_interval_cheb(num, deg):
    mat = numpy.zeros((deg+1, num+1))
    int_ = numpy.zeros((deg+1,))
    for i in range(deg+1):
        for j in range(num+1):
            theta = math.acos(2.0 * j / num - 1.0)
            val = math.cos(theta * i)
            mat[i, j] = val
    while len(cheb_list) < deg+1:
        cheb = (2*x * cheb_list[-1] - cheb_list[-2]).expand()
        int__ = float(sympy.integrate(cheb, (x, -1, 1)))
        cheb_list.append(cheb)
        cheb_int.append(int__)
    for i in range(deg+1):
        int_[i] = cheb_int[i]
    return mat, int_

In [ ]:
now_deg = 1
flag = False

deg_list = [0]

for num in range(1, 200):
    while True:
        mat, int_ = mat_interval_cheb(num, now_deg+2)
        res = scipy.optimize.linprog(
            numpy.zeros(num+1),
            -numpy.eye(num+1),
            numpy.zeros(num+1),
            mat,
            int_,
        )
        if res.success:
            now_deg += 2
        else:
            break
    deg_list.append(now_deg)

In [ ]:
print("\\hline")
text = " & ".join(str(i) for i in range(1, 11)) + " \\\\"
print(text)
print("\\hline")
text = " & ".join(str(i) for i in deg_list[1:11]) + " \\\\"
print(text)
print("\\hline")
text = " & ".join(str(i) for i in range(11, 21)) + " \\\\"
print(text)
print("\\hline")
text = " & ".join(str(i) for i in deg_list[11:21]) + " \\\\"
print(text)
print("\\hline")

In [ ]:
deg = 1
search_list = []
while True:
    pos = bisect.bisect_left(deg_list, deg)
    if pos < len(deg_list):
        search_list.append((deg, pos))
        deg += 2
    else:
        break

In [ ]:
print("\\hline")
text = " & ".join(str(i[0]) for i in search_list[:10]) + " \\\\"
print(text)
print("\\hline")
text = " & ".join(str(i[1]) for i in search_list[:10]) + " \\\\"
print(text)
print("\\hline")
text = " & ".join(str(i[0]) for i in search_list[10:20]) + " \\\\"
print(text)
print("\\hline")
text = " & ".join(str(i[1]) for i in search_list[10:20]) + " \\\\"
print(text)
print("\\hline")

In [ ]:
num = 16
deg = 11

In [ ]:
mat, int_ = mat_interval_cheb(num, deg)

In [ ]:
res = scipy.optimize.linprog(
    numpy.zeros(num+1),
    -numpy.eye(num+1),
    numpy.zeros(num+1),
    mat,
    int_,
    method="simplex"
)

In [ ]:
print(res.message)

In [ ]:
mat, int_ = mat_interval_direct(num, deg)

In [ ]:
error = mat.dot(res.x) - int_
print(numpy.linalg.norm(error))